In [1]:
import selenium
from selenium import webdriver
import time
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import selenium.webdriver.support.ui as UI
from tqdm import tqdm
import numpy as np

In [78]:
url = 'https://www.engelvoelkers.com/en-be/belgium/?gclid=Cj0KCQjwqrb7BRDlARIsACwGad6QajuajESE8Zx-MeIc2GPQv3E08aMio-L35qs59mkhmA-GlZeQUEMaAq09EALw_wcB'
driver=webdriver.Firefox()
driver.get(url)
driver.implicitly_wait(0.5)

In [79]:
pop_up_OK=driver.find_element_by_class_name('ev-btn').click()


In [80]:
dropdown_menu=driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[2]/div[1]/div[2]/div[2]/div[1]/div').click()
driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[2]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div/span').click()


In [32]:
search=driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[2]/div[1]/div[8]/button').click()

In [33]:
pop_agree=driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div/div[3]/button[2]').click()

In [34]:
def get_page_links(url,driver,pages=(550)):
    
    hrefs = []

    for i in range(pages):
        property_links = driver.find_elements_by_class_name('ev-property-container')
    
        for lnk in property_links:
            if lnk.get_attribute('href') == None:
                continue
            else:
                hrefs.append(lnk.get_attribute('href'))
     
        try:
        
            driver.find_element_by_class_name('ev-pager-next').click()
        except:
            print('done')
    return hrefs     

In [35]:
def get_html_data(driver,url):
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup
    

In [81]:
def get_location(soup):
    try:
        loc = soup.find(
            'div', class_='ev-exposee-content ev-exposee-subtitle').text.split(',')
        locality = loc[3]
        return locality
    except:
        return 'None'


def get_property_type(soup):
    try:
        prop = soup.find(
            'div', class_='ev-exposee-content ev-exposee-subtitle').text.split(',')
        property_type = prop[0]
        return property_type
    except:
        return 'None'


def get_subtype(soup):
    try:
        st = driver.find_element_by_xpath(
            '/html/body/div[7]/div[4]/ul[3]/li[1]/span').text
        sub_type = st
        return sub_type
    except:
        return 'None'


def get_price(soup):
    try:
        price = soup.find('span', attrs={'itemprop': 'price'}).text
        cost = price
        return cost
    except:
        return 'None'


def get_type_of_sale(soup):
    try:
        st = soup.find('div', attrs={
                       'class': 'ev-exposee-content ev-exposee-subtitle'}).text.split(',')
        s_type = st[0]
        return s_type
    except:
        return 'None'


def get_number_of_rooms(soup):
    tot = 0
    try:
        broom = driver.find_element_by_xpath(
            '/html/body/div[7]/div[4]/div[2]/div[2]/div[1]').text
        troom = driver.find_element_by_xpath(
            '/html/body/div[7]/div[4]/div[2]/div[3]/div[1]').text
        tot = int(broom)+int(troom)

        return tot
    except:
        return 'None'


def get_area(soup):
    try:
        area = driver.find_element_by_xpath(
            '/html/body/div[7]/div[4]/div[2]/div[4]/div[1]').text.split(' ')[0]
        return area
    except:
        return 'None'


def is_kitchen_equipped(soup):

    try:

        kit = driver.find_element_by_xpath(
            '/html/body/div[7]/div[4]/p').text.split(' ')

        if 'fitted' in kit:
            return 1
        else:
            return 0
    except:
        return 'None'


def is_furnished(soup):
    try:
        furn = soup.find('id')
        if furn:
            return furn
        else:
            return 'None'
    except:
        return 'None'


def is_open_fire(soup):
    try:
        fir = soup.find('id')
        if fir:
            return fir
        else:
            return 'None'
    except:
        return 'None'


def is_terrace(soup):
    try:
        terr = soup.find('id')
        if terr:
            return terr
        else:
            return 'None'
    except:
        return 'None'


def is_garden(soup):
    try:
        kit = driver.find_element_by_xpath(
            '/html/body/div[7]/div[4]/p').text.split(' ')

        if 'garden' in kit:
            return 1
        else:
            return 0

    except:
        return 'None'


def get_surface_land(soup):
    try:
        surf_land = soup.find('id')
        if surf_land:
            return surf_land
        else:
            return 'None'
    except:
        return 'None'


def get_surface_plot_land(soup):
    try:
        surface_plot = driver.find_element_by_xpath(
            '/html/body/div[7]/div[4]/div[2]/div[5]/div[1]').text.split(' ')[0]
        return surface_plot
    except:
        return 'None'


def get_facades(soup):
    try:
        facs = soup.find('id')
        if facs:
            return facs
        else:
            return 'None'
    except:
        return 'None'


def is_swim_pool(soup):
    try:
        kit = driver.find_element_by_xpath(
            '/html/body/div[7]/div[4]/p').text.split(' ')
        if 'swimming' in kit:
            return 1
        else:
            return 0
    except:
        return 'None'


def get_state_of_building(soup):
    try:
        prop = soup.find(
            'div', class_='ev-exposee-content ev-exposee-subtitle').text.split(',')
        property_type = prop[0]
        if 'Apartment' in property_type:
        
            year_const = driver.find_element_by_xpath(
            '/html/body/div[7]/div[4]/ul[3]/li[1]/span').text
        if 'House' in property_type:
            year_const = driver.find_element_by_xpath(
            '/html/body/div[7]/div[4]/ul[3]/li[2]/span').text
        if int(year_const) < 2000:
            return 'old'
        else:
            return 'New'
    except:
        return 'None'

    # loc=driver.find_element_by_xpath('/html/body/div[7]/div[4]/div[1]').text

In [37]:
from tqdm import tqdm
overall_data=[]
links=get_page_links(url,driver,pages=550)
print(len(links))


9898


In [82]:
for i in tqdm(links):
    
    soup=get_html_data(driver,i)
    location=get_location(soup)
    proptype=get_property_type(soup)
    subtype=get_subtype(soup)
    price=get_price(soup)
    type_of_sale=get_type_of_sale(soup)
    no_rooms=get_number_of_rooms(soup)
    area=get_area(soup)
    fully_equipped_kitchen=is_kitchen_equipped(soup)
    furnished=is_furnished(soup)
    open_fire=is_open_fire(soup)
    terrace_area=is_terrace(soup)
    garden_area=is_garden(soup)
    surface_land=get_surface_land(soup)
    surface_plot_land=get_surface_plot_land(soup)
    no_facades=get_facades(soup)
    swim_pool=is_swim_pool(soup)
    state_of_the_build=get_state_of_building(soup)
    
    
    property_info={'Locality':location,'Type of Property':proptype,
              'Sub Type':subtype,'Price':price,'Type of Sale':type_of_sale,
                   'No of Rooms':no_rooms,'Area(sqft)':area,'Equipped(Y/N)':fully_equipped_kitchen,
                   'Furnished(Y/N)':furnished,'Open Fire(Y/N)':open_fire,
                   'Terrace Area':terrace_area,'Garden Area':garden_area,
                   'Surface of the Land(sqft)':surface_land,
                   'Surface area of the Plot(sqft)':surface_plot_land,
                   'No of Facades': no_facades,'Simming Pool(Y/N)':swim_pool,
                   'State of the Building':state_of_the_build}
    overall_data.append(property_info)
    



  0%|          | 0/9898 [00:00<?, ?it/s]

  0%|          | 1/9898 [00:03<9:22:32,  3.41s/it]

  0%|          | 2/9898 [00:05<8:02:26,  2.93s/it]

  0%|          | 3/9898 [00:06<6:19:46,  2.30s/it]

  0%|          | 4/9898 [00:10<7:43:20,  2.81s/it]

  0%|          | 5/9898 [00:11<6:46:06,  2.46s/it]

  0%|          | 6/9898 [00:13<6:12:06,  2.26s/it]

  0%|          | 7/9898 [00:15<6:20:05,  2.31s/it]

  0%|          | 8/9898 [00:19<7:48:55,  2.84s/it]

  0%|          | 9/9898 [00:21<6:27:46,  2.35s/it]

  0%|          | 10/9898 [00:22<5:57:07,  2.17s/it]

  0%|          | 11/9898 [00:29<9:11:52,  3.35s/it]

  0%|          | 12/9898 [00:30<7:19:14,  2.67s/it]

  0%|          | 13/9898 [00:32<7:03:27,  2.57s/it]

  0%|          | 14/9898 [00:35<7:30:36,  2.74s/it]

  0%|          | 15/9898 [00:37<7:08:51,  2.60s/it]

  0%|          | 16/9898 [00:39<6:35:53,  2.40s/it]

  0%|          | 17/9898 [00:45<8:57:02,  3.26s/it]

  0%|          | 18/9898 [00:48<8:44:16,  3.18s/it]

  0%|      

  1%|▏         | 138/9898 [06:29<3:35:50,  1.33s/it]

  1%|▏         | 139/9898 [06:31<4:02:56,  1.49s/it]

  1%|▏         | 140/9898 [06:34<4:40:25,  1.72s/it]

  1%|▏         | 141/9898 [06:35<4:40:00,  1.72s/it]

  1%|▏         | 142/9898 [06:38<5:25:22,  2.00s/it]

  1%|▏         | 143/9898 [06:39<4:32:31,  1.68s/it]

  1%|▏         | 144/9898 [06:40<4:19:53,  1.60s/it]

  1%|▏         | 145/9898 [06:43<5:06:41,  1.89s/it]

  1%|▏         | 146/9898 [06:45<5:02:52,  1.86s/it]

  1%|▏         | 147/9898 [06:46<4:55:41,  1.82s/it]

  1%|▏         | 148/9898 [06:48<4:26:42,  1.64s/it]

  2%|▏         | 149/9898 [06:51<6:02:02,  2.23s/it]

  2%|▏         | 150/9898 [06:55<7:22:12,  2.72s/it]

  2%|▏         | 151/9898 [06:56<5:56:02,  2.19s/it]

  2%|▏         | 152/9898 [06:59<6:15:39,  2.31s/it]

  2%|▏         | 153/9898 [07:01<6:32:55,  2.42s/it]

  2%|▏         | 154/9898 [07:08<10:01:14,  3.70s/it]

  2%|▏         | 155/9898 [07:13<10:56:07,  4.04s/it]

  2%|▏         | 156/9898 

  3%|▎         | 274/9898 [12:33<7:42:44,  2.88s/it]

  3%|▎         | 275/9898 [12:36<8:26:54,  3.16s/it]

  3%|▎         | 276/9898 [12:39<7:50:51,  2.94s/it]

  3%|▎         | 277/9898 [12:41<6:59:25,  2.62s/it]

  3%|▎         | 278/9898 [12:49<11:27:43,  4.29s/it]

  3%|▎         | 279/9898 [12:51<10:05:28,  3.78s/it]

  3%|▎         | 280/9898 [12:54<8:45:59,  3.28s/it] 

  3%|▎         | 281/9898 [12:55<7:29:56,  2.81s/it]

  3%|▎         | 282/9898 [12:57<6:38:13,  2.48s/it]

  3%|▎         | 283/9898 [13:00<7:09:21,  2.68s/it]

  3%|▎         | 284/9898 [13:01<5:52:06,  2.20s/it]

  3%|▎         | 285/9898 [13:04<6:41:50,  2.51s/it]

  3%|▎         | 286/9898 [13:06<5:59:22,  2.24s/it]

  3%|▎         | 287/9898 [13:12<8:49:34,  3.31s/it]

  3%|▎         | 288/9898 [13:13<7:19:28,  2.74s/it]

  3%|▎         | 289/9898 [13:16<7:40:12,  2.87s/it]

  3%|▎         | 290/9898 [13:19<7:14:37,  2.71s/it]

  3%|▎         | 291/9898 [13:22<7:53:16,  2.96s/it]

  3%|▎         | 292/9898

  4%|▍         | 410/9898 [18:54<4:09:35,  1.58s/it]

  4%|▍         | 411/9898 [18:55<3:58:59,  1.51s/it]

  4%|▍         | 412/9898 [18:56<3:41:43,  1.40s/it]

  4%|▍         | 413/9898 [19:00<5:25:18,  2.06s/it]

  4%|▍         | 414/9898 [19:04<6:50:52,  2.60s/it]

  4%|▍         | 415/9898 [19:08<7:53:21,  3.00s/it]

  4%|▍         | 416/9898 [19:14<10:55:06,  4.15s/it]

  4%|▍         | 417/9898 [19:16<8:41:19,  3.30s/it] 

  4%|▍         | 418/9898 [19:18<7:41:36,  2.92s/it]

  4%|▍         | 419/9898 [19:22<8:59:06,  3.41s/it]

  4%|▍         | 420/9898 [19:23<7:07:08,  2.70s/it]

  4%|▍         | 421/9898 [19:25<5:56:57,  2.26s/it]

  4%|▍         | 422/9898 [19:29<7:49:20,  2.97s/it]

  4%|▍         | 423/9898 [19:30<6:08:52,  2.34s/it]

  4%|▍         | 424/9898 [19:31<5:17:13,  2.01s/it]

  4%|▍         | 425/9898 [19:34<6:00:43,  2.28s/it]

  4%|▍         | 426/9898 [19:36<5:16:59,  2.01s/it]

  4%|▍         | 427/9898 [19:37<4:53:29,  1.86s/it]

  4%|▍         | 428/9898 

  6%|▌         | 546/9898 [24:36<6:19:23,  2.43s/it]

  6%|▌         | 547/9898 [24:38<5:56:51,  2.29s/it]

  6%|▌         | 548/9898 [24:41<6:01:31,  2.32s/it]

  6%|▌         | 549/9898 [24:43<5:33:07,  2.14s/it]

  6%|▌         | 550/9898 [24:45<5:36:13,  2.16s/it]

  6%|▌         | 551/9898 [24:46<4:54:05,  1.89s/it]

  6%|▌         | 552/9898 [24:54<9:50:13,  3.79s/it]

  6%|▌         | 553/9898 [24:56<7:55:41,  3.05s/it]

  6%|▌         | 554/9898 [24:57<6:29:50,  2.50s/it]

  6%|▌         | 555/9898 [25:02<8:52:43,  3.42s/it]

  6%|▌         | 556/9898 [25:04<7:08:09,  2.75s/it]

  6%|▌         | 557/9898 [25:06<6:51:03,  2.64s/it]

  6%|▌         | 558/9898 [25:12<9:26:21,  3.64s/it]

  6%|▌         | 559/9898 [25:15<8:45:58,  3.38s/it]

  6%|▌         | 560/9898 [25:17<7:51:06,  3.03s/it]

  6%|▌         | 561/9898 [25:23<10:33:32,  4.07s/it]

  6%|▌         | 562/9898 [25:26<9:06:18,  3.51s/it] 

  6%|▌         | 563/9898 [25:28<8:02:31,  3.10s/it]

  6%|▌         | 564/9898 

...

 27%|██▋       | 2647/9898 [2:04:39<5:53:31,  2.93s/it]

 27%|██▋       | 2648/9898 [2:04:41<5:16:51,  2.62s/it]

 27%|██▋       | 2649/9898 [2:04:44<5:43:52,  2.85s/it]

 27%|██▋       | 2650/9898 [2:04:47<5:42:15,  2.83s/it]

 27%|██▋       | 2651/9898 [2:04:52<6:49:36,  3.39s/it]

 27%|██▋       | 2652/9898 [2:04:54<6:01:07,  2.99s/it]

 27%|██▋       | 2653/9898 [2:04:56<5:24:12,  2.68s/it]

 27%|██▋       | 2654/9898 [2:05:03<8:06:57,  4.03s/it]

 27%|██▋       | 2655/9898 [2:05:08<9:01:41,  4.49s/it]

 27%|██▋       | 2656/9898 [2:05:13<8:51:12,  4.40s/it]

 27%|██▋       | 2657/9898 [2:05:19<9:53:21,  4.92s/it]

 27%|██▋       | 2658/9898 [2:05:23<9:20:17,  4.64s/it]

 27%|██▋       | 2659/9898 [2:05:25<7:39:12,  3.81s/it]

 27%|██▋       | 2660/9898 [2:05:29<7:48:03,  3.88s/it]

 27%|██▋       | 2661/9898 [2:05:30<6:14:26,  3.10s/it]

 27%|██▋       | 2662/9898 [2:05:34<6:56:29,  3.45s/it]

 27%|██▋       | 2663/9898 [2:05:36<6:00:41,  2.99s/it]

 27%|██▋       | 2664/9898 [2:0

 28%|██▊       | 2777/9898 [2:11:25<4:25:17,  2.24s/it]

 28%|██▊       | 2778/9898 [2:11:29<5:45:06,  2.91s/it]

 28%|██▊       | 2779/9898 [2:11:31<4:53:01,  2.47s/it]

 28%|██▊       | 2780/9898 [2:11:33<4:54:11,  2.48s/it]

 28%|██▊       | 2781/9898 [2:11:35<4:43:42,  2.39s/it]

 28%|██▊       | 2782/9898 [2:11:37<4:12:01,  2.13s/it]

 28%|██▊       | 2783/9898 [2:11:38<3:38:34,  1.84s/it]

 28%|██▊       | 2784/9898 [2:11:40<3:38:55,  1.85s/it]

 28%|██▊       | 2785/9898 [2:11:42<4:04:43,  2.06s/it]

 28%|██▊       | 2786/9898 [2:11:48<6:04:55,  3.08s/it]

 28%|██▊       | 2787/9898 [2:11:50<5:39:37,  2.87s/it]

 28%|██▊       | 2788/9898 [2:11:51<4:38:29,  2.35s/it]

 28%|██▊       | 2789/9898 [2:11:54<4:55:52,  2.50s/it]

 28%|██▊       | 2790/9898 [2:11:55<4:07:40,  2.09s/it]

 28%|██▊       | 2791/9898 [2:11:58<4:32:14,  2.30s/it]

 28%|██▊       | 2792/9898 [2:12:00<4:12:00,  2.13s/it]

 28%|██▊       | 2793/9898 [2:12:02<3:57:49,  2.01s/it]

 28%|██▊       | 2794/9898 [2:1

 29%|██▉       | 2906/9898 [2:17:56<6:44:10,  3.47s/it]

 29%|██▉       | 2907/9898 [2:17:58<6:15:58,  3.23s/it]

 29%|██▉       | 2908/9898 [2:18:00<5:30:01,  2.83s/it]

 29%|██▉       | 2909/9898 [2:18:05<6:49:03,  3.51s/it]

 29%|██▉       | 2910/9898 [2:18:08<6:24:30,  3.30s/it]

 29%|██▉       | 2911/9898 [2:18:14<7:37:51,  3.93s/it]

 29%|██▉       | 2912/9898 [2:18:16<6:45:52,  3.49s/it]

 29%|██▉       | 2913/9898 [2:18:17<5:22:10,  2.77s/it]

 29%|██▉       | 2914/9898 [2:18:22<6:38:13,  3.42s/it]

 29%|██▉       | 2915/9898 [2:18:24<5:27:05,  2.81s/it]

 29%|██▉       | 2916/9898 [2:18:29<7:11:58,  3.71s/it]

 29%|██▉       | 2917/9898 [2:18:31<5:49:29,  3.00s/it]

 29%|██▉       | 2918/9898 [2:18:32<5:05:57,  2.63s/it]

 29%|██▉       | 2919/9898 [2:18:35<5:11:12,  2.68s/it]

 30%|██▉       | 2920/9898 [2:18:37<4:53:46,  2.53s/it]

 30%|██▉       | 2921/9898 [2:18:39<4:26:32,  2.29s/it]

 30%|██▉       | 2922/9898 [2:18:41<4:24:47,  2.28s/it]

 30%|██▉       | 2923/9898 [2:1

 31%|███       | 3035/9898 [2:24:41<4:47:06,  2.51s/it]

 31%|███       | 3036/9898 [2:24:43<4:18:18,  2.26s/it]

 31%|███       | 3037/9898 [2:24:44<3:55:47,  2.06s/it]

 31%|███       | 3038/9898 [2:24:50<6:09:18,  3.23s/it]

 31%|███       | 3039/9898 [2:24:55<6:55:56,  3.64s/it]

 31%|███       | 3040/9898 [2:25:02<9:09:40,  4.81s/it]

 31%|███       | 3041/9898 [2:25:06<8:45:23,  4.60s/it]

 31%|███       | 3042/9898 [2:25:08<6:56:34,  3.65s/it]

 31%|███       | 3043/9898 [2:25:11<6:47:51,  3.57s/it]

 31%|███       | 3044/9898 [2:25:14<6:08:30,  3.23s/it]

 31%|███       | 3045/9898 [2:25:21<8:14:17,  4.33s/it]

 31%|███       | 3046/9898 [2:25:23<7:09:23,  3.76s/it]

 31%|███       | 3047/9898 [2:25:30<8:41:06,  4.56s/it]

 31%|███       | 3048/9898 [2:25:33<7:50:41,  4.12s/it]

 31%|███       | 3049/9898 [2:25:35<7:02:38,  3.70s/it]

 31%|███       | 3050/9898 [2:25:38<6:14:38,  3.28s/it]

 31%|███       | 3051/9898 [2:25:39<5:19:22,  2.80s/it]

 31%|███       | 3052/9898 [2:2

TimeoutException: Message: Timeout loading page after 300000ms


In [83]:
df=pd.DataFrame(overall_data,columns=['Locality','Type of Property',
              'Sub Type','Price','Type of Sale',
                   'No of Rooms','Area(sqft)','Equipped(Y/N)',
                   'Furnished(Y/N)','Open Fire(Y/N)',
                   'Terrace Area','Garden Area',
                   'Surface of the Land(sqft)',
                   'Surface area of the Plot(sqft)',
                   'No of Facades','Simming Pool(Y/N)',
                   'State of the Building'])
with open("Engel_Voelkers_dataset.csv","a") as f:
    df.to_csv(f)